# Data Processing
#### Θα ξεκινήσουμε να κάνουμε process τα business data.

In [1]:
import pandas as pd
import numpy as np


In [2]:
business_json_path = 'yelp_dataset/yelp_academic_dataset_business.json'
df = pd.read_json(business_json_path, lines=True)
df.shape

(150346, 14)

In [3]:
df_valid = df.loc[df['city'] == "Philadelphia"]
df_valid.shape

(14569, 14)

In [4]:
df_valid=df_valid.dropna().reset_index(drop=True)
df_valid.shape

(11070, 14)

In [5]:
df_valid.to_csv("business.csv")


In [6]:
df_bus = pd.read_csv("business.csv")
df_bus.shape


(11070, 15)

Θα φτιάξουμε ένα set με όλα τα business ids, ώστε να μπορέσουμε να κάνουμε το φιλτράρισμα των reviews, κοιτάζοντας αν το business_id του review βρίσκεται μεσα στο business_id_set. Ο κάθε τέτοιος έλεγχος γίνεται περίπου σε O(1) χρόνο λόγο της δομής δεδομένων set.  (Εδώ και στην Ερώτηση3.Ζ είναι τα μόνα σημεία που έχω χρησιμοποιήσει set)

In [7]:
business_id_set = set(df_bus.business_id)

print(len(business_id_set))


11070


Ως αναφορα τα reviews, δεν μπορουμε να τα βαλουμε σε Dataframe γιατι ειναι πολλα και θα πρεπει πρωτα να τα φιτραρουμε διαβαζωντας τα γραμμη-γραμμη.
Θελουμε να κρατησουμε μονο αυτα που το business_id τους, ειναι στην λιστα με τα busness_id(στα παραπανω) 

In [8]:
import json
review_json_path = 'yelp_dataset/yelp_academic_dataset_review.json'



In [9]:
f = open(review_json_path, 'r')
review_business_id_list = []

count = 0
while True:
    try:
        line = f.readline()
        if line[88: 110] in business_id_set:# business_id_set:
            review_business_id_list.append(line)
        if line == "":
            break
    
    except:
        continue
    
print(len(review_business_id_list))
f.close()

881973


Σε αυτο το σημειο εχουμε μια λιστα review_business_id_list που περιεχει ολα τα reviews που αντιστοιχουν στις επιχειρησεις που συγκεντρωσαμε παραπανω business.json. Τωρα, θελουμε να φερουμε τα reviews σε καταλληλη μορφη ωστε να βγαζουν νοημα στο csv.
Για αυτο αρχικα θα χωρισω το καθε review στα ," που δηλωνη την εισαγωγη του επομενου πεδιου.
Στην συνεχεια, θελουμε να αποσπασουμε το information μεσα απο καθε ζευγαρι (attribute:information) ετσι ωστε να βαλουμε την καταλληλη τιμη κατω απο καθε header του csv.

In [10]:
lst = review_business_id_list[0].split(",\"")
import csv
with open("reviews.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']) 
    attr_list = []
    for review in review_business_id_list:
        review_list = review.split(",\"")

        for attribute in review_list:
            try:
                if attribute == '...' or attribute=='':
                    continue
                
                info = attribute.split(":",1)[1]
                if info[0] == '"':
                    attr_list.append(info.strip().strip('}"'))
                else:
                    attr_list.append(info) 
            except:
                print(review)
                print("///////////////")
                print(review_list)
                print("///////////////")
                print(attribute)
                raise Exception("--------------------------")
        writer.writerow(attr_list)
        attr_list = []


In [11]:
df_rev = pd.read_csv('reviews.csv')


In [12]:
print(df_rev.shape)
df_rev.head()

(881973, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
2,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
3,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
4,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0,0,0,Tremendous service (Big shout out to Douglas) ...,2013-06-24 11:21:25


## Τέλος | Data Processing
Σε αυτό το σημείο έχουμε τελειώσει με το Data Processing, έχοντας δημιουργήσει δύο csv αρχεία, business.csv και review.csv και ξεκινάμε να απαντάμε στα ερωτήματα της άσκησης.
